In [1]:
import pandas as pd
import numpy as np

import re
from itertools import chain
from collections import Counter

from google.cloud import bigquery
from google.oauth2 import service_account

from datetime import datetime, timedelta

import requests
import warnings
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# InsecureRequestWarning 경고 무시
warnings.simplefilter('ignore', InsecureRequestWarning)

In [2]:
KEY_PATH = ".config/"
servicekey_path = KEY_PATH + "serviceKey.json" ## 빅쿼리 외 다른 API 활용 위해
bigquerykey_path = KEY_PATH + "mido-project-426906-31b49963ac97.json"

warnings.filterwarnings("ignore")

In [3]:
# BigQuery 클라이언트 생성 함수
def create_bigquery_client(key_path):
    credentials = service_account.Credentials.from_service_account_file(key_path)
    client = bigquery.Client(credentials=credentials, project=credentials.project_id)
    return client

In [4]:
def save_dataframe_to_bigquery(df, dataset_id, table_id, key_path):
    # BigQuery 클라이언트 객체 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 데이터프레임을 BigQuery 테이블에 적재
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = "WRITE_TRUNCATE"  # 기존 테이블 내용 삭제 후 삽입

    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()  # 작업 완료 대기

    print(f"Data inserted into table {table_id} successfully.")

In [5]:
def get_dataframe_from_bigquery(dataset_id, table_id, key_path):
    # BigQuery 클라이언트 생성
    client = create_bigquery_client(key_path)

    # 테이블 레퍼런스 생성
    table_ref = client.dataset(dataset_id).table(table_id)

    # 테이블 데이터를 DataFrame으로 변환
    df = client.list_rows(table_ref).to_dataframe()

    return df

In [6]:
# 오늘 날짜
today = datetime.today()#.strftime('%Y%m%d')

# 어제 날짜 계산
ytday = datetime.today() - timedelta(days=1)

# 만약 어제, 오늘이 토요일(5) 또는 일요일(6)이라면, 그 전주 금요일로 변경
if ytday.weekday() == 5:  # 토요일
    ytday -= timedelta(days=1)
elif ytday.weekday() == 6:  # 일요일
    ytday -= timedelta(days=2)
if today.weekday() == 5:  # 토요일
    today -= timedelta(days=1)
elif today.weekday() == 6:  # 일요일
    today -= timedelta(days=2)

# 'YYYYMMDD' 형식으로 변환
ytday = ytday.strftime('%Y%m%d')
today = today.strftime('%Y%m%d')

#### 법정동코드

In [31]:
dist_code = pd.read_csv('C:\py_src\mido\data/법정동코드 전체자료.csv',encoding='cp949')

In [32]:
dist_code_list = []
for i in range(dist_code['법정동명'].str.split().apply(len).max()):
    dist_code_temp = dist_code[dist_code['법정동명'].str.split().str[i].str[-1].isin(['읍','면','동','리','가','로']).dropna()]
    dist_code_list.append([dist_code_temp['법정동명'].str.split().str[i].dropna().unique()])

emd_code_list = list(chain(*chain(*dist_code_list)))

#### 종합쇼핑몰 납품상세내역

In [8]:
all_shop_df = get_dataframe_from_bigquery('g2b', 'shop_detail_df_all', bigquerykey_path)

In [9]:
all_shop_df_fin = all_shop_df[['납품요구접수일자', '수요기관명', '납품요구건명', '업체명', '단가', '단위', '수량', '금액', '수요기관코드', '수요기관구분', '수요기관지역명','납품요구지청명']]
all_shop_df_fin = all_shop_df_fin.sort_values(['납품요구접수일자'],ascending=False).reset_index(drop=True)

In [10]:
# 특수문자, 숫자, 영어 제거 함수 (연속된 공백을 단일 공백으로 변환 포함)
def clean_text(text):
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', ' ', text)
    text = re.sub('\s+', ' ', text).strip()  # 연속된 공백을 단일 공백으로 변환
    return text

In [11]:
# 사업명 전처리
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명'].apply(clean_text)
# all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.strip()

all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
all_shop_df_fin['납품요구건명_re'] = all_shop_df_fin['납품요구건명_re'].apply((' ').join)

In [12]:
# 수요기관 필터링
dist_nm = pd.Series(all_shop_df_fin[all_shop_df_fin['수요기관지역명'].str.split(' ').str[1].notnull()]['수요기관지역명'].str.split(' ').str[1].unique())
dist_nm = np.where(dist_nm.apply(len)<=2,dist_nm,dist_nm.str[:-1])

In [13]:
# 사업명 split 키워드 필터링
mapping_keywd_all = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' '))) ## 전체 키워드
mapping_keywd_all_filter_cnt = pd.Series(Counter(mapping_keywd_all))

mapping_keywd3 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:3])) ## split 3개 키워드
mapping_keywd3_filter_cnt = pd.Series(Counter(mapping_keywd3))

mapping_keywd2 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:2])) ## split 2개 키워드
mapping_keywd2_filter_cnt = pd.Series(Counter(mapping_keywd2))

mapping_keywd1 = list(chain(*all_shop_df_fin['납품요구건명_re'].str.split(' ').str[:1])) ## split 1개 키워드
mapping_keywd1_filter_cnt = pd.Series(Counter(mapping_keywd1))

mapping_keywd_all_filter_cnt_nm = ('|').join(mapping_keywd_all_filter_cnt[mapping_keywd_all_filter_cnt <= mapping_keywd_all_filter_cnt.mean()].keys())
mapping_keywd3_filter_cnt_nm = ('|').join(mapping_keywd3_filter_cnt[mapping_keywd3_filter_cnt <= mapping_keywd3_filter_cnt.mean()].keys())
mapping_keywd2_filter_cnt_nm = ('|').join(mapping_keywd2_filter_cnt[mapping_keywd2_filter_cnt <= mapping_keywd2_filter_cnt.mean()].keys())
mapping_keywd1_filter_cnt_nm = ('|').join(mapping_keywd1_filter_cnt[mapping_keywd1_filter_cnt <= mapping_keywd1_filter_cnt.mean()].keys())

In [14]:
# 전체 키워드 데이터셋
mapping_keywd_df = pd.DataFrame(Counter(mapping_keywd_all).items(),columns=['키워드','빈도수'])
mapping_keywd_df = mapping_keywd_df[mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
1,인조잔디,815
7,관급자재,459
9,구입,255
2,구매,138
11,운동장,97
...,...,...
582,거로마을,1
583,꿈나무근린공원,1
585,부안초,1
587,바닥개선,1


In [15]:
# 종합쇼핑몰 기준 중요키워드 추출
print('1등급 키워드 : ', list(mapping_keywd_df[mapping_keywd_df['빈도수'] >= 50].sort_values('빈도수',ascending=False)['키워드']))
print('2등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 50) & (mapping_keywd_df['빈도수'] >= 40)].sort_values('빈도수',ascending=False)['키워드']))
print('3등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 40) & (mapping_keywd_df['빈도수'] >= 30)].sort_values('빈도수',ascending=False)['키워드']))
print('4등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 30) & (mapping_keywd_df['빈도수'] >= 20)].sort_values('빈도수',ascending=False)['키워드']))
print('5등급 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 20) & (mapping_keywd_df['빈도수'] >= 10)].sort_values('빈도수',ascending=False)['키워드']))
print('기타 키워드 : ', list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)].sort_values('빈도수',ascending=False)['키워드']))

1등급 키워드 :  ['인조잔디', '관급자재', '구입', '구매', '운동장', '조성공사', '공사', '테니스장', '관급', '조성사업', '게이트볼장', '체육시설', '정비공사', '교체공사']
2등급 키워드 :  []
3등급 키워드 :  ['조성', '설치', '축구장', '풋살장', '개선공사']
4등급 키워드 :  ['파크골프장', '족구장', '설치공사', '교체', '토목', '조달구매']
5등급 키워드 :  ['환경개선', '보수공사', '정비사업', '환경개선공사', '다목적구장', '건축', '개보수공사', '시설개선', '개선사업', '시설개선사업', '시설개선공사', '잔디', '연병장', '개선', '보수', '부대', '다목적', '체육공원', '조달']
기타 키워드 :  ['사업', '생활체육공원', '신축공사', '조경', '생활체육시설', '교체사업', '관련', '정비', '어린이공원', '개소', '추가', '신축', '물품', '개보수', '시설', '야구장', '단계', '따른', '위한', '노후시설', '김영훈', '체육', '차분', '리모델링', '인조잔디운동장', '당진', '관급자', '공공하수처리시설', '증설공사', '실내게이트볼장', '창녕국민체육센터', '구입의뢰', '치수과', '해운대수목원', '트랙조성', '시행', '구장', '요청', '그라운드골프장', '본부', '여단', '우수조달', '설치사업', '사단', '관급자관급', '건립', '배드민턴장', '비가림시설', '하부', '일원', '확장', '대보수', '재정비공사', '학교운동장', '조경공사', '근린공원', '재정비사업', '보조경기장', '트랙보수', '체육시설관리과', '조달구입', '환경친화적', '환경개선사업', '구매설치', '인조잔디외', '자재', '본원', '수해복구공사', '수지아르피아', '배수판', '휀스', '운동공간', '운동장환경개선', '그라운드', '재조성', '파크골프연습장', '시설물', 

In [16]:
# 빈도수 키워드
freq_kwd = ('|').join(list(mapping_keywd_df[(mapping_keywd_df['빈도수'] < 10)]['키워드']))

#### 지자체 세부사업별 예산서

In [17]:
# aaa = pd.read_csv('C:\py_src\midoproject\data/2024-07-07T22-12_export.csv')

In [18]:
# budget_df_today = budget_df_today[budget_df_today['세부사업명'].isin(aaa['세부사업명'])]

In [53]:
# 빅쿼리에서 불러오기
budget_df_today = get_dataframe_from_bigquery('budget','budget_df_0' + today,bigquerykey_path)
budget_df_ytday = get_dataframe_from_bigquery('budget','budget_df_0' + ytday,bigquerykey_path)

In [54]:
# 필요키워드1
need_kwd1 = '학교|초등학교|중학교|고등학교|대학교|다목적|운동장|종합운동장|공원|체육공원|체육시설|체육센터|스포츠센터|연병장|축구장|족구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장|관급'

# 필요키워드2
need_kwd2 = '개선|교체|보수|공사|구매|구입|설치|정비|조성'

# 필요키워드3
need_kwd3 = '인조잔디'

# 제외키워드
except_kwd = '경관|조명|펜스|복지|교육|학습|보건|행사|예방|안전|치안|창출|활성화|유아|검진|조명|오염|예술|급식|보상|주차장|개방|CCTV|미술|교실|관리|운영|자전거|울타리|버스|맨홀|노점|노상|하천|하수도|광고|약수터|전통시장|계단|정화조|도로확장|골프장|입학|도로개설|소프트웨어|가속기|진로진학|인터넷|음악|취업|역량|억제제|살포|급수관|하수관로|요양기관|도로방음벽|가로등|도로개설|보행자도로|진입도로|철거'

In [55]:
budget_df_today = budget_df_today[((budget_df_today['세부사업명'].str.contains(need_kwd1) & 
                                  budget_df_today['세부사업명'].str.contains(need_kwd2)) &
                                  ~budget_df_today['세부사업명'].str.contains(except_kwd)) |
                                  budget_df_today['세부사업명'].str.contains(need_kwd3)
                                  ].reset_index(drop=True)

In [56]:
# # 키워드 필터링
# budget_df_today = budget_df_today[budget_df_today['세부사업명'].str.contains(need_kwd1) & ~budget_df_today['세부사업명'].str.contains(except_kwd)].reset_index(drop=True)
# budget_df_ytday = budget_df_ytday[budget_df_ytday['세부사업명'].str.contains(need_kwd1) & ~budget_df_ytday['세부사업명'].str.contains(except_kwd)].reset_index(drop=True)

In [57]:
# 종료 사업
budget_df_delete_temp1 = budget_df_ytday[budget_df_ytday['세부사업코드'].isin(set(budget_df_ytday['세부사업코드']) - set(budget_df_today['세부사업코드']))].reset_index(drop=True)
budget_df_delete_temp2 = budget_df_ytday[budget_df_ytday['세부사업명'].isin(set(budget_df_ytday['세부사업명']) - set(budget_df_today['세부사업명']))].reset_index(drop=True)
budget_df_delete = pd.concat([budget_df_delete_temp1,budget_df_delete_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_delete

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드
0,2024,1100000,서울,1122000,서울은평구,일반회계,3110000202430157,공원 내 폭염저감시설 설치,20240705,124000000,0,124000000,국토및지역개발,지역및도시,3110000


In [58]:
# 새로 추가된 사업
budget_df_new_temp1 = budget_df_today[~budget_df_today['세부사업코드'].isin(budget_df_ytday['세부사업코드'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new_temp2 = budget_df_today[~budget_df_today['세부사업명'].isin(budget_df_ytday['세부사업명'])].reset_index(drop=True).reset_index(drop=True)
budget_df_new = pd.concat([budget_df_new_temp1,budget_df_new_temp2],axis=0).drop_duplicates().reset_index(drop=True)
budget_df_new

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,지출액,편성액,분야명,부문명,행정자치단체코드
0,2024,1100000,서울,1132000,서울서초구,일반회계,3210000202430098,말죽거리근린공원 정비사업,20240708,400000000,0,400000000,환경,자연,3210000


In [59]:
# 사업명 전처리
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명'].apply(clean_text)
# budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.strip()

budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].str.split().apply(lambda x: [kw for kw in x if len(kw) > 1]) ## 글자수 1 제외
budget_df_today['세부사업명_re'] = budget_df_today['세부사업명_re'].apply((' ').join)

In [60]:
# 해당 지역 추출
filtered_budget_df_temp1 = budget_df_today[budget_df_today['자치단체명'].str.contains(('|').join(dist_nm))].reset_index(drop=True)
filtered_budget_df_temp2 = budget_df_today[budget_df_today['자치단체명'].str.contains('본청')].reset_index(drop=True)

filtered_budget_df_today = pd.concat([filtered_budget_df_temp1,filtered_budget_df_temp2],axis=0).reset_index(drop=True)

#### 지자체 종합쇼핑몰 전처리

In [61]:
# Initialize the dictionary for split keywords
shop_split_kwd = {}
for i in range(all_shop_df_fin['납품요구건명_re'].str.split().apply(len).max()):
    shop_split_kwd['kwd_' + str(i)] = all_shop_df_fin['납품요구건명_re'].str.split().str[i]

In [62]:
# 종합쇼핑몰 납품요구건명 split 키워드 순서대로 맵핑
# Initialize the list to store the results
mapping_merge_df = []

# Iterate over each keyword column
for i in range(len(shop_split_kwd)):
    keyword_column = 'kwd_' + str(i)
    
    # Add the new keyword columns to the dataframes
    filtered_budget_df_today[keyword_column] = ''
    all_shop_df_fin[keyword_column] = ''
    
    # Get unique non-null keywords
    shop_split_kwd_temp = shop_split_kwd[keyword_column].dropna().unique()
    
    # Update the keyword columns based on the presence of keywords
    for kwd in shop_split_kwd_temp:
        filtered_budget_df_today[keyword_column] = np.where(
            filtered_budget_df_today['세부사업명_re'].str.contains(kwd, na=False),
            filtered_budget_df_today[keyword_column] + ' ' + kwd,
            filtered_budget_df_today[keyword_column]
        )
        all_shop_df_fin[keyword_column] = np.where(
            all_shop_df_fin['납품요구건명_re'].str.contains(kwd, na=False),
            all_shop_df_fin[keyword_column] + ' ' + kwd,
            all_shop_df_fin[keyword_column]
        )
    
    filtered_budget_df_today[keyword_column] = filtered_budget_df_today[keyword_column].str.strip()
    all_shop_df_fin[keyword_column] = all_shop_df_fin[keyword_column].str.strip()

    # Create a temporary DataFrame with matched keywords
    mapping_merge_df_temp = pd.merge(
        filtered_budget_df_today[(filtered_budget_df_today[keyword_column] != '') & ~(filtered_budget_df_today[keyword_column].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', keyword_column]].reset_index(drop=True),
        all_shop_df_fin[(all_shop_df_fin[keyword_column] != '') & ~(all_shop_df_fin[keyword_column].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', keyword_column]].reset_index(drop=True),
        how='left',
        on=keyword_column
        ).drop_duplicates().reset_index(drop=True)
    
    # Append the temporary DataFrame to the results list
    mapping_merge_df.append(mapping_merge_df_temp)

In [63]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 추출
# Initialize the columns for district keywords
filtered_budget_df_today['kwd_sig'] = ''
all_shop_df_fin['kwd_sig'] = ''

# Update the district keyword columns based on the presence of district names
for dist in dist_nm:
    filtered_budget_df_today['kwd_sig'] = np.where(
        filtered_budget_df_today['세부사업명_re'].str.contains(dist, na=False),
        filtered_budget_df_today['kwd_sig'] + ' ' + dist,
        filtered_budget_df_today['kwd_sig']
    )
    all_shop_df_fin['kwd_sig'] = np.where(
        all_shop_df_fin['납품요구건명_re'].str.contains(dist, na=False),
        all_shop_df_fin['kwd_sig'] + ' ' + dist,
        all_shop_df_fin['kwd_sig']
    )
    
# Create a DataFrame for district keywords
filtered_budget_df_today['kwd_sig'] = filtered_budget_df_today['kwd_sig'].str.strip()
filtered_budget_df_today['kwd_sig'] = filtered_budget_df_today['kwd_sig'].str.split()
filtered_budget_df_today = filtered_budget_df_today.explode('kwd_sig')

all_shop_df_fin['kwd_sig'] = all_shop_df_fin['kwd_sig'].str.strip()
all_shop_df_fin['kwd_sig'] = all_shop_df_fin['kwd_sig'].str.split()
all_shop_df_fin = all_shop_df_fin.explode('kwd_sig')

mapping_merge_df_sig = pd.merge(
    filtered_budget_df_today[(filtered_budget_df_today['kwd_sig'] != '') & ~(all_shop_df_fin[keyword_column].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_sig']].reset_index(drop=True),
    all_shop_df_fin[(all_shop_df_fin['kwd_sig'] != '') & ~(all_shop_df_fin[keyword_column].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_sig']].reset_index(drop=True),
    how='left',
    on='kwd_sig'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_sig)

In [65]:
# 종합쇼핑몰 지역 키워드 맵핑 - 세부사업명 지역 및 해당지역 읍면동 추출
# Initialize the columns for district keywords
filtered_budget_df_today['kwd_emd'] = ''
all_shop_df_fin['kwd_emd'] = ''

# Update the district keyword columns based on the presence of district names
for emd in set(emd_code_list):
    filtered_budget_df_today['kwd_emd'] = np.where(
        filtered_budget_df_today['세부사업명_re'].str.contains(emd, na=False),
        filtered_budget_df_today['kwd_emd'] + ' ' + emd,
        filtered_budget_df_today['kwd_emd']
    )
    all_shop_df_fin['kwd_emd'] = np.where(
        all_shop_df_fin['납품요구건명_re'].str.contains(emd, na=False),
        all_shop_df_fin['kwd_emd'] + ' ' + emd,
        all_shop_df_fin['kwd_emd']
    )
    all_shop_df_fin['kwd_emd'] = all_shop_df_fin['kwd_emd'].str.strip()

filtered_budget_df_today['kwd_emd'] = filtered_budget_df_today['kwd_emd'].str.strip()
filtered_budget_df_today['kwd_emd'] = filtered_budget_df_today['kwd_emd'].str.split()
filtered_budget_df_today = filtered_budget_df_today.explode('kwd_emd')

all_shop_df_fin['kwd_emd'] = all_shop_df_fin['kwd_emd'].str.strip()
all_shop_df_fin['kwd_emd'] = all_shop_df_fin['kwd_emd'].str.split()
all_shop_df_fin = all_shop_df_fin.explode('kwd_emd')

# Create a DataFrame for district keywords
mapping_merge_df_emd = pd.merge(
    filtered_budget_df_today[(filtered_budget_df_today['kwd_emd'] != '') & ~(filtered_budget_df_today['kwd_emd'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_emd']].reset_index(drop=True),
    all_shop_df_fin[(all_shop_df_fin['kwd_emd'] != '') & ~(all_shop_df_fin['kwd_emd'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_emd']].reset_index(drop=True),
    how='left',
    on='kwd_emd'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_emd)

In [75]:
# 필요키워드 앞부분 추출
need_kwd_split = need_kwd1.split('|')

need_kwd_split_temp = []
for need_split in need_kwd_split:

    shop_need_split = all_shop_df_fin['납품요구건명_re'].str.split(need_split).str[0].str.split().str[-1].dropna().unique()
    need_kwd_split_temp.append(shop_need_split)

need_kwd_list = list(pd.Series(list(chain(*need_kwd_split_temp))).apply(clean_text))

In [83]:
all_shop_df_fin['납품요구건명_re'].apply(lambda x: re.split(need_kwd1,x)).apply(len)

0      2
1      3
2      2
3      3
4      3
      ..
899    4
900    3
901    4
902    3
903    3
Name: 납품요구건명_re, Length: 973, dtype: int64

In [79]:
all_shop_df_fin['납품요구건명_re'].apply(lambda x: [re.split(need_kwd1, part)[0] for part in x.split()])

0                                [운산, 인조잔디, 구매]
1                  [창녕국민, 시설개선, 공사, 토목, , 인조잔디]
2                                  [, 인조잔디, 구입]
3                                  [문현, , 인조잔디]
4                                  [도계늑구, 인조잔디]
                         ...                   
899           [내항, 생활, , 보수공사, , 인조잔디, 체육, 김영훈]
900                   [파주중, 인조잔디, , 조성사업, , 구입]
901                            [배정미래, , 조성공사, ]
902                      [건국, 인조잔디조성공사, , 인조잔디]
903    [원곡, 친환경, 인조잔디, , 조성, 사업을, 위한, 조달물품, 구입]
Name: 납품요구건명_re, Length: 973, dtype: object

In [78]:
need_kwd1

'학교|초등학교|중학교|고등학교|대학교|다목적|운동장|종합운동장|공원|체육공원|체육시설|체육센터|스포츠센터|연병장|축구장|족구장|야구장|풋살장|배드민턴장|테니스장|게이트볼장|하키장|관급'

In [41]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 앞부분 매핑
# Initialize the columns for district keywords
filtered_budget_df_today['kwd_need_split'] = ''
all_shop_df_fin['kwd_need_split'] = ''

# Update the district keyword columns based on the presence of district names
for need_split in need_kwd_list:
    filtered_budget_df_today['kwd_need_split'] = np.where(
        filtered_budget_df_today['세부사업명_re'].str.contains(need_split, na=False),
        filtered_budget_df_today['kwd_need_split'] + ' ' + need_split,
        filtered_budget_df_today['kwd_need_split']
    )
    filtered_budget_df_today['kwd_need_split'] = filtered_budget_df_today['kwd_need_split'].str.strip()

    all_shop_df_fin['kwd_need_split'] = np.where(
        all_shop_df_fin['납품요구건명_re'].str.contains(need_split, na=False),
        all_shop_df_fin['kwd_need_split'] + ' ' + need_split,
        all_shop_df_fin['kwd_need_split']
    )
    all_shop_df_fin['kwd_need_split'] = all_shop_df_fin['kwd_need_split'].str.strip()

# Create a DataFrame for district keywords
mapping_merge_df_need_split = pd.merge(
    filtered_budget_df_today[(filtered_budget_df_today['kwd_need_split'] != '') & ~(filtered_budget_df_today['kwd_need_split'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need_split']].reset_index(drop=True),
    all_shop_df_fin[(all_shop_df_fin['kwd_need_split'] != '') & ~(filtered_budget_df_today['kwd_need_split'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need_split']].reset_index(drop=True),
    how='left',
    on='kwd_need_split'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need_split)

In [43]:
# 종합쇼핑몰 지역 키워드 맵핑 - 필요키워드 매핑
# Initialize the columns for district keywords
filtered_budget_df_today['kwd_need'] = ''
all_shop_df_fin['kwd_need'] = ''

# 필요키워드 추출
need_kwd = need_kwd1.split('|')

# Update the district keyword columns based on the presence of district names
for need in need_kwd:
    filtered_budget_df_today['kwd_need'] = np.where(
        filtered_budget_df_today['세부사업명_re'].str.contains(need, na=False),
        filtered_budget_df_today['kwd_need'] + ' ' + need,
        filtered_budget_df_today['kwd_need']
    )
    filtered_budget_df_today['kwd_need'] = filtered_budget_df_today['kwd_need'].str.strip()

    all_shop_df_fin['kwd_need'] = np.where(
        all_shop_df_fin['납품요구건명_re'].str.contains(need, na=False),
        all_shop_df_fin['kwd_need'] + ' ' + need,
        all_shop_df_fin['kwd_need']
    )
    all_shop_df_fin['kwd_need'] = all_shop_df_fin['kwd_need'].str.strip()

# Create a DataFrame for district keywords
mapping_merge_df_need = pd.merge(
    filtered_budget_df_today[(filtered_budget_df_today['kwd_need'] != '') & ~(filtered_budget_df_today['kwd_need'].isnull())][['지역명','자치단체명','세부사업코드','세부사업명', '세부사업명_re', 'kwd_need']].reset_index(drop=True),
    all_shop_df_fin[(all_shop_df_fin['kwd_need'] != '') & ~(all_shop_df_fin['kwd_need'].isnull())][['수요기관명','수요기관지역명','납품요구건명', '납품요구건명_re', 'kwd_need']].reset_index(drop=True),
    how='left',
    on='kwd_need'
    ).drop_duplicates().reset_index(drop=True)

# Append the district keywords DataFrame to the results list
mapping_merge_df.append(mapping_merge_df_need)

In [44]:
# 키워드별 데이터셋 병합
mapping_merge_df_fin = pd.concat(mapping_merge_df, axis=0).drop_duplicates()
mapping_merge_df_fin = mapping_merge_df_fin[~mapping_merge_df_fin['납품요구건명'].isnull()].reset_index(drop=True)

In [46]:
# 모든 키워드 합
kwd_columns = ['kwd_0','kwd_1','kwd_2','kwd_3','kwd_4','kwd_5','kwd_6','kwd_7','kwd_8','kwd_sig','kwd_emd','kwd_need_split','kwd_need']
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin[kwd_columns].apply(lambda x : (' ').join(x.dropna()),axis=1)
mapping_merge_df_fin['all_kwd'] = mapping_merge_df_fin['all_kwd'].str.strip()
mapping_merge_df_fin.drop(columns=kwd_columns[:-4], inplace=True)

In [47]:
# 지역명 추출1
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[0]

# Define the replacements
replacements = {
    '전라북도': '전북',
    '전라남도': '전남',
    '경상북도': '경북',
    '경상남도': '경남',
    '충청북도': '충북',
    '충청남도': '충남'
}

# Apply the replacements
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].replace(replacements)
mapping_merge_df_fin['수요기관지역명_split1'] = mapping_merge_df_fin['수요기관지역명_split1'].str[:2]

In [48]:
# 지역명 추출2
mapping_merge_df_fin['자치단체명_re'] = ''

mapping_merge_df_fin['수요기관지역명_split2'] = mapping_merge_df_fin['수요기관지역명'].str.split().str[1] ## 수요기관지역명 split 2번째 추출

# Update the district keyword columns based on the presence of district names
for dist2 in mapping_merge_df_fin['수요기관지역명_split2'].dropna().unique():
    mapping_merge_df_fin['자치단체명_re'] = np.where(
        mapping_merge_df_fin['자치단체명'].str.contains(dist2, na=False),
        mapping_merge_df_fin['자치단체명_re'] + ' ' + dist2,
        mapping_merge_df_fin['자치단체명_re']
    )

mapping_merge_df_fin['자치단체명_re'] = mapping_merge_df_fin['자치단체명_re'].str.strip()

In [50]:
# 제외 지역 제거
filtered_mapping_merge_df_fin = mapping_merge_df_fin[mapping_merge_df_fin['자치단체명_re']!=''].drop_duplicates().reset_index(drop=True)

In [51]:
# 지역 일치 확인1
mapping_merge_df_fin_temp1 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['지역명']==filtered_mapping_merge_df_fin['수요기관지역명_split1']].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명'].str.contains('|'.join(mapping_merge_df_fin_temp1['수요기관지역명_split2']))].drop_duplicates().reset_index(drop=True)
mapping_merge_df_fin_temp1 = mapping_merge_df_fin_temp1[mapping_merge_df_fin_temp1['자치단체명_re']==mapping_merge_df_fin_temp1['수요기관지역명_split2']].drop_duplicates().reset_index(drop=True)

# 본청 포함 확인
mapping_merge_df_fin_temp2 = filtered_mapping_merge_df_fin[filtered_mapping_merge_df_fin['자치단체명'].str.contains('본청')].drop_duplicates().reset_index(drop=True)

# 데이터 병합
mapping_merge_df_final = pd.concat([mapping_merge_df_fin_temp1,mapping_merge_df_fin_temp2], axis=0).drop_duplicates().reset_index(drop=True)

In [52]:
mapping_merge_df_final

,지역명,자치단체명,세부사업코드,세부사업명,세부사업명_re,수요기관명,수요기관지역명,납품요구건명,납품요구건명_re,kwd_sig,kwd_emd,kwd_need_split,kwd_need,all_kwd,수요기관지역명_split1,자치단체명_re,수요기관지역명_split2
0,대구,대구동구,3420000202430036,신암4동 가로수 보호판 인조잔디 설치사업(주민참여예산 동참여형 사업),신암 가로수 보호판 인조잔디 설치사업 주민참여예산 동참여형 사업,한국부동산원,대구광역시 동구,인조잔디 발주,인조잔디 발주,NaN,NaN,NaN,NaN,인조잔디,대구,동구,동구
1,경기,경기수원시,37400002023304BE,만석공원 게이트볼장 시설개선,만석공원 게이트볼장 시설개선,경기도 수원시 장안구,경기도 수원시 장안구,만석공원 게이트볼장 정비(시설개선) 공사,만석공원 게이트볼장 정비 시설개선 공사,NaN,NaN,NaN,NaN,만석공원 게이트볼장,경기,수원시,수원시
2,경기,경기수원시,3740000202430181,만석공원 게이트볼장 정비,만석공원 게이트볼장 정비,경기도 수원시 장안구,경기도 수원시 장안구,만석공원 게이트볼장 정비(시설개선) 공사,만석공원 게이트볼장 정비 시설개선 공사,NaN,NaN,NaN,NaN,만석공원 게이트볼장,경기,수원시,수원시
3,경기,경기성남시,3780000201630024,황송공원인조잔디구장 운영관리사업(공사 위탁사업),황송공원인조잔디구장 운영관리사업 공사 위탁사업,경기도 성남시 푸른도시사업소,경기도 성남시 중원구,인조잔디 구입-자혜근린공원 망덕배드민턴장 및 운동기구 재정비공사,인조잔디 구입 자혜근린공원 망덕배드민턴장 운동기구 재정비공사,NaN,NaN,NaN,NaN,인조잔디,경기,성남시,성남시
4,경기,경기평택시,3910000202330601,포승레포츠공원 축구장 인조잔디 교체(도비),포승레포츠공원 축구장 인조잔디 교체 도비,경기도 평택시,경기도 평택시,포승레포츠공원 축구장 인조잔디 교체공사-인조잔디,포승레포츠공원 축구장 인조잔디 교체공사 인조잔디,NaN,NaN,NaN,NaN,인조잔디 축구장 포승레포츠공원,경기,평택시,평택시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,경남,경남양산시,53800002024302BB,삼호어린이공원 시설개선사업,삼호어린이공원 시설개선사업,양산시시설관리공단,경상남도 양산시,공원시설관리팀 인조잔디 수선(잔디구입),공원시설관리팀 인조잔디 수선 잔디구입,NaN,NaN,NaN,공원,공원,경남,양산시,양산시
1029,경남,경남산청군,5450000202330005,장승배기 생태공원 체육시설 정비사업(전환사업),장승배기 생태공원 체육시설 정비사업 전환사업,경상남도 산청군,경상남도 산청군,장승배기 생태공원 체육시설 - 인조잔디,장승배기 생태공원 체육시설 인조잔디,NaN,NaN,NaN,공원 체육시설,공원 체육시설,경남,산청군,산청군
1030,경남,경남산청군,5450000200830034,도시자연공원 정비,도시자연공원 정비,경상남도 산청군,경상남도 산청군,삼장 여울공원 파크골프장 조성사업 - 인조잔디,삼장 여울공원 파크골프장 조성사업 인조잔디,NaN,NaN,NaN,공원,공원,경남,산청군,산청군
1031,경남,경남함양군,54600002023301DB,함양읍 게이트볼장 지붕 교체 및 설치사업(2023년 특별조정교부금),함양읍 게이트볼장 지붕 교체 설치사업 특별조정교부금,경상남도 함양군 병곡면,경상남도 함양군,물품 구입(등구정 게이트볼장 인조잔디 교체사업),물품 구입 등구정 게이트볼장 인조잔디 교체사업,NaN,NaN,NaN,게이트볼장,게이트볼장,경남,함양군,함양군


In [86]:
# Use re.split to split the strings by multiple delimiters
mapping_merge_df_final['납품요구건명_키워드'] = mapping_merge_df_final['납품요구건명_re'].apply(lambda x: re.split(need_kwd1, x)).str[0].str.split().str[-1]
mapping_merge_df_final['세부사업명_키워드'] = mapping_merge_df_final['세부사업명_re'].apply(lambda x: re.split(need_kwd1, x)).str[0].str.split().str[-1]

In [79]:
filtered_budget_df_today[filtered_budget_df_today['세부사업명'].isin(mapping_merge_df_final['세부사업명'])]

,회계연도,지역코드,지역명,자치단체코드,자치단체명,회계구분명,세부사업코드,세부사업명,집행일자,예산현액,...,kwd_3,kwd_4,kwd_5,kwd_6,kwd_7,kwd_8,kwd_sig,kwd_emd,kwd_need_split,kwd_need
10,2024,1100000,서울,1117000,서울중랑구,일반회계,3060000201530079,중랑구립잔디운동장 시설개선,20240708,179091000,...,시설개선 운동장 개선 시설,잔디 운동장 시설 시설개선,운동장 잔디 시설,잔디,잔디,,중랑,,잔디 잔디 잔디 잔디 중 잔디 잔디 잔디 잔디 잔디 잔디 잔디 운동장 잔디 잔디 ...,운동장
41,2024,1100000,서울,1134000,서울송파구,일반회계,3230000202330323,문정공원 시설정비사업,20240708,577523000,...,사업 정비사업 시설,정비사업 정비 사업 시설정비사업 시설,정비사업 시설 정비 사업,,,,,,정비사업 사업 정비사업 사업 정비사업 사업 정비사업 사업 정비사업 사업 정비사업 ...,공원
42,2024,1100000,서울,1134000,서울송파구,일반회계,3230000202330381,송파근린공원 재조성사업,20240708,700000000,...,조성 조성사업 사업 재조성,조성사업 조성 사업,조성사업 사업,조성사업,,,송파,,조성사업 사업 조성 조성사업 사업 조성 조성사업 사업 조성 조성사업 사업 조성 조...,공원
54,2024,2600000,부산,2621000,부산금정구,일반회계,3350000202230106,생활체육시설 조성,20240708,1199177200,...,조성 체육시설 시설,조성 체육 시설,시설,체육,체육,,,,조성 조성 조성 조성 조성 조성 조성 조성 생활체육 체육 조성 생활 조성 생활 조...,체육시설
55,2024,2600000,부산,2621000,부산금정구,일반회계,3350000202330188,구민운동장 환경개선,20240708,250000000,...,환경개선 운동장 개선,운동장,운동장,,,,,,운동장 운동장 환경개선 환경개선 개선 운동장,운동장
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
993,2024,3200000,세종,3200000,세종본청,일반회계,569000020143005E,도시공원 조성,20240708,6200000,...,조성,조성,,,,,,,조성 조성 조성 조성 조성 조성 조성 조성 조성 조성 조성 조성 조성 조성 조성 ...,공원
999,2024,4400000,충남,4400000,충남본청,일반회계,64400002023303F3,체육시설 조성,20240708,500000000,...,조성 체육시설 시설,조성 체육 시설,시설,체육,체육,,,,조성 조성 조성 조성 조성 조성 조성 조성 체육 조성 조성 조성 조성 조성 조성 ...,체육시설
1000,2024,4500000,전북,4500000,전북본청,일반회계,64500002016302EA,공공체육시설 개보수 지원,20240708,4170000000,...,체육시설 보수 시설,체육 시설,시설 개보수,체육,체육,,,,체육 공공 체육시설 체육시설 체육시설 보수 개보수,체육시설
1001,2024,4600000,전남,4600000,전남본청,일반회계,64600002019302BF,공공체육시설 개보수 지원,20240708,2658000000,...,체육시설 보수 시설,체육 시설,시설 개보수,체육,체육,,,,체육 공공 체육시설 체육시설 체육시설 보수 개보수,체육시설


In [77]:
mapping_merge_df_final['세부사업명']

0                   황송공원인조잔디구장 운영관리사업(공사 위탁사업)
1                  체육진흥시설지원(금일 다목적구장 조성)(전환사업)
2              체육진흥시설지원(체육공원 레저스포츠 시설조성)(전환사업)
3                     고매동 체육시설 개선공사[특별조정교부금사업]
4                     영덕동 체육시설 개선공사[특별조정교부금사업]
                        ...                   
276                          고령우곡면게이트볼장시설물보수공사
277                                    체육시설 조성
278                                동네체육시설 유지보수
279                                  체육시설 유지보수
280    백전면 전천후 게이트볼장 태양광설비 설치사업(2024년 특별조정교부금)
Name: 세부사업명, Length: 281, dtype: object

In [106]:
mapping_merge_df_final.to_csv('C:\py_src\midoproject\data/mapping_df_temp.csv',encoding='utf-8-sig',index=False)

In [52]:
aa = pd.read_csv('C:\py_src\midoproject\data/aa.csv')

In [53]:
mapping_merge_df_final[mapping_merge_df_final['납품요구건명'].isin(aa['납품요구건명'].unique())]

,지역명,자치단체명,세부사업코드,세부사업명,세부사업명_re,수요기관명,수요기관지역명,납품요구건명,납품요구건명_re,kwd_sig,kwd_emd,kwd_need_split,kwd_need,all_kwd,수요기관지역명_split1,자치단체명_re,수요기관지역명_split2
1,경기,경기수원시,37400002023304BE,만석공원 게이트볼장 시설개선,만석공원 게이트볼장 시설개선,경기도 수원시 장안구,경기도 수원시 장안구,만석공원 게이트볼장 정비(시설개선) 공사,만석공원 게이트볼장 정비 시설개선 공사,NaN,NaN,NaN,NaN,만석공원 게이트볼장,경기,수원시,수원시
2,경기,경기수원시,3740000202430181,만석공원 게이트볼장 정비,만석공원 게이트볼장 정비,경기도 수원시 장안구,경기도 수원시 장안구,만석공원 게이트볼장 정비(시설개선) 공사,만석공원 게이트볼장 정비 시설개선 공사,NaN,NaN,NaN,NaN,만석공원 게이트볼장,경기,수원시,수원시
3,경기,경기성남시,3780000201630024,황송공원인조잔디구장 운영관리사업(공사 위탁사업),황송공원인조잔디구장 운영관리사업 공사 위탁사업,경기도 성남시 푸른도시사업소,경기도 성남시 중원구,인조잔디 구입-자혜근린공원 망덕배드민턴장 및 운동기구 재정비공사,인조잔디 구입 자혜근린공원 망덕배드민턴장 운동기구 재정비공사,NaN,NaN,NaN,NaN,인조잔디,경기,성남시,성남시
4,경기,경기평택시,3910000202330601,포승레포츠공원 축구장 인조잔디 교체(도비),포승레포츠공원 축구장 인조잔디 교체 도비,경기도 평택시,경기도 평택시,포승레포츠공원 축구장 인조잔디 교체공사-인조잔디,포승레포츠공원 축구장 인조잔디 교체공사 인조잔디,NaN,NaN,NaN,NaN,인조잔디 축구장 포승레포츠공원,경기,평택시,평택시
5,경기,경기용인시,405000020233041E,기흥동 게이트볼장 인조잔디 교체,기흥동 게이트볼장 인조잔디 교체,경기도 용인시,경기도 용인시 처인구,기흥동 게이트볼장 인조잔디 교체공사-보차도용콘크리트블록,기흥동 게이트볼장 인조잔디 교체공사 보차도용콘크리트블록,NaN,NaN,NaN,NaN,인조잔디 게이트볼장 기흥동,경기,용인시,용인시
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
811,경남,경남양산시,5380000202330837,삽량체육공원 시설 개선공사,삽량체육공원 시설 개선공사,경상남도 양산시,경상남도 양산시,웅상체육공원 정비공사 - 인조잔디,웅상체육공원 정비공사 인조잔디,NaN,NaN,NaN,공원 체육공원,공원 체육공원,경남,양산시,양산시
812,경남,경남양산시,53800002024301AD,웅상체육공원 솔밭황톳길 조성,웅상체육공원 솔밭황톳길 조성,경상남도 양산시,경상남도 양산시,웅상체육공원 정비공사 - 인조잔디,웅상체육공원 정비공사 인조잔디,NaN,NaN,NaN,공원 체육공원,공원 체육공원,경남,양산시,양산시
813,경남,경남양산시,538000020243021B,동면 상동체육공원 정비공사,동면 상동체육공원 정비공사,경상남도 양산시,경상남도 양산시,웅상체육공원 정비공사 - 인조잔디,웅상체육공원 정비공사 인조잔디,NaN,NaN,NaN,공원 체육공원,공원 체육공원,경남,양산시,양산시
847,경남,경남양산시,53800002024302A3,상삼체육공원 환경개선사업,상삼체육공원 환경개선사업,경상남도 양산시,경상남도 양산시,웅상체육공원 정비공사 - 인조잔디,웅상체육공원 정비공사 인조잔디,NaN,NaN,NaN,공원 체육공원,공원 체육공원,경남,양산시,양산시


In [54]:
aa[aa['납품요구건명'].isin(mapping_merge_df_final['납품요구건명'].unique())]

,집행일자,지역명,자치단체명,세부사업명,예산현액,국비,시도비,시군구비,기타,지출액,편성액,쇼핑몰,키워드,수요기관명,납품요구건명
12,20240708,부산광역시,북구,화명고가교 고가하부 체육시설 개보수,749760000,0,0,0,749760000,663022170,0,False,화명고가교 고가하부 체육시설,부산광역시 북구,"/화명고가교 고가하부 체육시설(족구장, 테니스장) 개보수 공사 관급자재("
15,20240708,대구광역시,달성군,화원 본리리 족구장 조성,189830000,0,0,189830000,0,29370590,178000000,False,화원 본리리 족구장,대구광역시 달성군,화원 본리리 족구장 조성 공사-인조잔디
16,20240708,대구광역시,달성군,현풍 테니스장 정비,602551100,0,0,602551100,0,0,0,False,현풍 테니스장,대구광역시 달성군,현풍 테니스장 정비 공사-인조잔디
17,20240708,대구광역시,달성군,화원게이트볼장 정원 조성 사업,485088000,0,0,485088000,0,455953960,0,False,화원게이트볼장,대구광역시 달성군,화원게이트볼장 정원 조성공사-인조잔디
21,20240708,인천광역시,강화군,포6중대 풋살장 조성공사,350000000,0,0,350000000,0,287494600,350000000,False,포6중대 풋살장,인천광역시 강화군,포6중대 풋살장 조성공사-인조잔디
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,20240708,경상남도,김해시,진영공설운동장 시설물 개선공사(전환사업),980860000,0,280860000,700000000,0,970089940,600000000,False,진영공설운동장,경상남도 김해시,진영공설운동장 정비공사 관급자재(인조잔디) 구입
138,20240708,경상남도,양산시,웅상체육공원 정비,982060000,982060000,0,0,0,604961970,0,False,웅상체육공원,경상남도 양산시,웅상체육공원 정비공사 - 인조잔디
139,20240708,경상남도,양산시,웅상체육공원 솔밭황톳길 조성,300000000,0,0,300000000,0,155674710,300000000,False,웅상체육공원,경상남도 양산시,웅상체육공원 정비공사 - 인조잔디
141,20240708,경상남도,산청군,장승배기 생태공원 체육시설 정비사업(전환사업),450000000,0,135000000,315000000,0,110136000,450000000,False,장승배기 생태공원,경상남도 산청군,장승배기 생태공원 체육시설 - 인조잔디


In [92]:
len(aa['납품요구건명'].unique())

125

In [87]:
aa

,집행일자,지역명,자치단체명,세부사업명,예산현액,국비,시도비,시군구비,기타,지출액,편성액,쇼핑몰,키워드,수요기관명,납품요구건명
0,20240708,서울특별시,성동구,살곶이체육공원 시설 전면 개선,4603629000,0,3700000000,903629000,0,2241894070,0,False,살곶이체육공원,서울특별시 성동구,"치수과-4983,박해나,인조잔디 구매(살곶이체육공원시설 전면 개선공사)"
1,20240708,서울특별시,성동구,응봉체육공원 테니스장 인조잔디 조성 등 환경개선,400000000,0,0,400000000,0,143524260,0,False,응봉체육공원,서울특별시 성동구,"토목과-5211,나병윤,인조잔디(응봉체육공원 테니스장 환경개선공사)"
2,20240708,서울특별시,강북구,삼각산중학교 옆길 산책로 정비(구 주민참여),150000000,0,0,150000000,0,9488000,150000000,False,삼각산중학교,서울특별시교육청 서울특별시성북교육지원청 삼각산중학교,삼각산중학교 운동장 인조잔디공사 관급자재 구입
3,20240708,서울특별시,노원구,중평어린이공원 정비,200000000,0,0,200000000,0,0,0,False,중평어린이공원,서울특별시 노원구,관급자재 구매[중평어린이공원-인조잔디]
4,20240708,서울특별시,노원구,중평어린이공원 조성,839524040,0,0,839524040,0,266337810,0,False,중평어린이공원,서울특별시 노원구,관급자재 구매[중평어린이공원-인조잔디]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,20240708,경상남도,양산시,웅상체육공원 솔밭황톳길 조성,300000000,0,0,300000000,0,155674710,300000000,False,웅상체육공원,경상남도 양산시,웅상체육공원 정비공사 - 인조잔디
140,20240708,경상남도,고성군,사회인 야구장 조성사업(전환사업),1823312270,0,0,1823312270,0,1734171440,0,False,사회인 야구장,경상남도 고성군,사회인 야구장 조성공사 관급자재(인조잔디)
141,20240708,경상남도,산청군,장승배기 생태공원 체육시설 정비사업(전환사업),450000000,0,135000000,315000000,0,110136000,450000000,False,장승배기 생태공원,경상남도 산청군,장승배기 생태공원 체육시설 - 인조잔디
142,20240708,경상남도,함양군,함양읍 게이트볼장 지붕 교체 및 설치사업(2023년 특별조정교부금),400000000,0,0,0,400000000,283048440,0,False,함양읍 게이트볼장,경상남도 함양군,관급자재 (함양읍 게이트볼장 지붕교체 및 설치사업)-인조잔디


In [23]:
# split 키워드 필터링
filtered_budget_df_today_temp1 = filtered_budget_df_today[filtered_budget_df_today['세부사업명_re'].str.contains(mapping_keywd_all_filter_cnt_nm)] ## 전체 키워드
filtered_budget_df_today_temp1 = filtered_budget_df_today_temp1[filtered_budget_df_today_temp1['세부사업명_re'].str.contains(mapping_keywd3_filter_cnt_nm)] ## split 3개 키워드
filtered_budget_df_today_temp1 = filtered_budget_df_today_temp1[filtered_budget_df_today_temp1['세부사업명_re'].str.contains(mapping_keywd2_filter_cnt_nm)] ## split 2개 키워드
filtered_budget_df_today_temp1 = filtered_budget_df_today_temp1[filtered_budget_df_today_temp1['세부사업명_re'].str.contains(mapping_keywd1_filter_cnt_nm)] ## split 1개 키워드

filtered_budget_df_today_temp2 = filtered_budget_df_today[filtered_budget_df_today['세부사업명'].str.contains(mapping_keywd_all_filter_cnt_nm)] ## 전체 키워드
filtered_budget_df_today_temp2 = filtered_budget_df_today_temp2[filtered_budget_df_today_temp2['세부사업명'].str.contains(mapping_keywd3_filter_cnt_nm)] ## split 3개 키워드
filtered_budget_df_today_temp2 = filtered_budget_df_today_temp2[filtered_budget_df_today_temp2['세부사업명'].str.contains(mapping_keywd2_filter_cnt_nm)] ## split 2개 키워드
filtered_budget_df_today_temp2 = filtered_budget_df_today_temp2[filtered_budget_df_today_temp2['세부사업명'].str.contains(mapping_keywd1_filter_cnt_nm)] ## split 1개 키워드

filtered_budget_df_today_fin = pd.concat([filtered_budget_df_today_temp1,filtered_budget_df_today_temp2],axis=0).drop_duplicates().reset_index(drop=True)

In [24]:
# 종합쇼핑몰 기준 필터링
bid_cd_split_today = list(budget_df_today[budget_df_today['세부사업코드'].isin(filtered_budget_df_today_fin['세부사업코드'])]['세부사업코드']) # split 키워드 필터링
bid_cd_freq_today = list(budget_df_today[budget_df_today['세부사업명_re'].str.contains(freq_kwd)]['세부사업코드']) + list(budget_df_today[budget_df_today['세부사업명'].str.contains(freq_kwd)]['세부사업코드'])# 빈도수 키워드 필터링

In [25]:
# 종합쇼핑몰 진행건 제외 & 지출액0(미진행건) 추가
budget_df_today_temp1 = budget_df_today[~budget_df_today['세부사업코드'].isin(set(bid_cd_split_today + bid_cd_freq_today))]
budget_df_today_temp2 = budget_df_today[budget_df_today['세부사업코드'].isin(set(bid_cd_split_today + bid_cd_freq_today))]
budget_df_today_temp2 = budget_df_today_temp2[budget_df_today_temp2['지출액']==0].reset_index(drop=True)
budget_df_today_fin = pd.concat([budget_df_today_temp1, budget_df_today_temp2], axis=0).drop_duplicates().reset_index(drop=True)

In [26]:
# 중요 사업 체크
budget_df_today_final = pd.concat([budget_df_today_fin[budget_df_today_fin['세부사업명'].str.contains('인조잔디')],
                                   budget_df_today_fin[~budget_df_today_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

# budget_df_ytday_final = pd.concat([budget_df_ytday_fin[budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')],
#                                    budget_df_ytday_fin[~budget_df_ytday_fin['세부사업명'].str.contains('인조잔디')]],axis=0).reset_index(drop=True)

In [27]:
# 최종데이터셋
budget_df_today_final = budget_df_today_final[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_delete = budget_df_delete[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)
budget_df_new = budget_df_new[['회계연도', '집행일자', '지역명', '자치단체명', '회계구분명', '세부사업명','예산현액', '지출액', '편성액', '분야명', '부문명']].drop_duplicates().reset_index(drop=True)

In [28]:
budget_df_today_final

,회계연도,집행일자,지역명,자치단체명,회계구분명,세부사업명,예산현액,지출액,편성액,분야명,부문명
0,2024,20240705,서울,서울구로구,일반회계,안양천 족구장 인조잔디 조성,30508440,0,0,문화및관광,체육
1,2024,20240705,대구,대구북구,일반회계,가로수 보호 인조잔디 매트 설치,22400000,0,22400000,국토및지역개발,지역및도시
2,2024,20240705,경기,경기수원시,일반회계,세류중학교 인조잔디운동장 시설 개선(도비),250000000,0,250000000,문화및관광,체육
3,2024,20240705,경기,경기의왕시,일반회계,한국철도공사 인재개발원 인조잔디구장 조성공사,1000000000,0,0,문화및관광,체육
4,2024,20240705,경기,경기이천시,수질개선특별회계,신둔면 체육공원 풋살장 및 족구장 인조잔디 설치(보조),289766000,0,289766000,문화및관광,체육
...,...,...,...,...,...,...,...,...,...,...,...
2038,2024,20240705,제주,제주본청,일반회계,서귀포추모공원 자연장지 확장 사업,1790000000,0,1790000000,사회복지,노인ㆍ청소년
2039,2024,20240705,제주,제주본청,일반회계,성산읍 테니스장 확충공사,100000000,0,100000000,문화및관광,체육
2040,2024,20240705,제주,제주본청,일반회계,장기미집행 도시계획시설(제1-55호 어린이공원) 조성사업,670000000,0,670000000,국토및지역개발,지역및도시
2041,2024,20240705,제주,제주본청,일반회계,장기미집행 도시계획시설(제1-57호 어린이공원) 조성사업,760000000,0,760000000,국토및지역개발,지역및도시


In [29]:
# 빅쿼리에 적재
save_dataframe_to_bigquery(budget_df_today_final,'budget','budget_df_listup',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_delete,'budget','budget_df_delete',bigquerykey_path)
save_dataframe_to_bigquery(budget_df_new,'budget','budget_df_new',bigquerykey_path)

Data inserted into table budget_df_listup successfully.
Data inserted into table budget_df_delete successfully.
Data inserted into table budget_df_new successfully.


#### 교육청 예산서

In [388]:
# 종합쇼핑몰 학교 관련 사업
shop_edu_mapping_temp1 = all_shop_df_fin[all_shop_df_fin['납품요구건명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_temp2 = all_shop_df_fin[all_shop_df_fin['수요기관명'].str.contains('학교|교육|교육청|교육지원청|교육부|공립|사립|국립')]
shop_edu_mapping_df = pd.concat([shop_edu_mapping_temp1,shop_edu_mapping_temp2],axis=0).drop_duplicates().reset_index(drop=True)
shop_edu_mapping_df

,납품요구접수일자,수요기관명,납품요구건명,업체명,단가,단위,수량,금액,수요기관코드,수요기관구분,수요기관지역명,납품요구지청명,납품요구건명_re
0,2024-07-03,인천광역시남부교육청 인성초등학교,인성초등학교 운동장 인조잔디 교체,케이앤비준우(주),73000,㎡,788.0,57524000,7320148,기타기관,인천광역시 중구,인천지방조달청,인성 운동장 인조잔디 교체
1,2024-07-03,서울특별시동부교육청 상봉중학교,상봉중학교 운동장 환경개선관련 관급자재 구매,주식회사 액션필드,60700,㎡,2597.0,157637900,7020148,교육기관,서울특별시 중랑구,서울지방조달청,상봉 운동장 환경개선관련 관급자재 구매
2,2024-07-03,선린대학,"선린대학교 운동장 환경개선(인조잔디, 트랙 등) 공사 관급자재 발주",주식회사 네오필드,74500,㎡,4890.0,364305000,7003548,기타기관,경상북도 포항시 북구,대구지방조달청,선린대학교 운동장 환경개선인조잔디 트랙 등 공사 관급자재 발주
3,2024-07-02,경기도용인교육청 용동중학교,용동중학교 운동장개선 공사 인조잔디 구입,케이앤비준우(주),39500,㎡,3600.0,142200000,7750018,기타기관,경기도 용인시 처인구,인천지방조달청,용동 운동장개선 공사 인조잔디 구입
4,2024-07-01,충청남도교육청 충남삼성고등학교,충남삼성고등학교 운동장 인조잔디 구매,주식회사 플랜에이,63200,㎡,3070.0,194024000,8140366,기타기관,충청남도 아산시,대전지방조달청,충남삼성 운동장 인조잔디 구매
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2024-01-09,서울특별시강동교육청 서울가동초등학교,실내운동장 인조잔디 구입,주식회사 필드글로벌,63000,㎡,495.0,31185000,7070052,교육기관,서울특별시 송파구,서울지방조달청,실내운동장 인조잔디 구입
225,2024-01-08,제주특별자치도교육청 제주시교육지원청 동광초등학교,동광초 인조잔디 및 울타리 조성공사 관급자재 구입(인조잔디 외 1종),주식회사 필드글로벌,63000,㎡,2680.0,168840000,9296060,교육기관,제주특별자치도 제주시,제주지방조달청,동광초 인조잔디 및 울타리 조성공사 관급자재 구입인조잔디 외 종
226,2024-01-04,한국과학기술원부설한국과학영재학교,풋살장 조성 및 기타공사 관급자재 구매,주식회사 이원,38000,㎡,747.0,28386000,B552516,기타공공기관,부산광역시 부산진구,부산지방조달청,풋살장 조성 및 기타공사 관급자재 구매
227,2024-01-03,경기도파주교육청 파주중학교,파주중 인조잔디 운동장 조성사업 관급자재 구입,주식회사 지에스케이,61500,㎡,2540.0,156210000,7680017,교육기관,경기도 파주시,서울지방조달청,파주중 인조잔디 운동장 조성사업 관급자재 구입


In [389]:
# 교육청, 교육지원청, 학교명 필터링
shop_edu_mapping_df['수요기관명_re'] = shop_edu_mapping_df['수요기관명'].str.replace('교육청','').str.replace('교육지원청','')
filter_edu1 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[0].notnull()]['수요기관명_re'].str.split().str[0].unique())
filter_edu2 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[1].notnull()]['수요기관명_re'].str.split().str[1].unique())
filter_edu3 = list(shop_edu_mapping_df[shop_edu_mapping_df['수요기관명_re'].str.split().str[2].notnull()]['수요기관명_re'].str.split().str[2].unique())

In [390]:
# 전체 키워드 데이터셋
edu_mapping_keywd_all = list(chain(*shop_edu_mapping_df['납품요구건명_re'].str.split(' '))) ## 전체 split 키워드
edu_mapping_keywd_df = pd.DataFrame(Counter(edu_mapping_keywd_all).items(),columns=['키워드','빈도수'])
edu_mapping_keywd_df = edu_mapping_keywd_df[edu_mapping_keywd_df['키워드'].apply(len)!=1].reset_index(drop=True)
edu_mapping_keywd_df.sort_values('빈도수',ascending=False)

,키워드,빈도수
2,인조잔디,130
1,운동장,84
15,구입,77
7,구매,64
55,관급자재인조잔디,63
...,...,...
164,토목공사,1
162,화산,1
161,목포부주,1
160,인조잔디구매,1


In [391]:
# 빅쿼리에서 불러오기
edu_budget_df = get_dataframe_from_bigquery('edu','edu_budget',bigquerykey_path)
edu_budget_df.sort_values('시도').reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행
0,강원특별자치도,화천교육지원청,공립,상서중학교 운동장 보수,"89,533,000","4,664",학교
1,강원특별자치도,인제교육지원청,공립,상남중학교 개방형 운동장 조성,"725,302,000","6,507",교육청
2,강원특별자치도,강원특별자치도교육청,공립,강릉제일고등학교 축구장 인조잔디 교체,"1,020,560,000","8,576",교육청
3,강원특별자치도,강릉교육지원청,공립,관동중학교 인조잔디 운동장 조성,"806,400,000","4,082",교육청
4,강원특별자치도,강원특별자치도교육청,사립,동해광희고등학교 인조잔디운동장 보수,"51,700,000",None,None
...,...,...,...,...,...,...,...
189,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None
190,충청남도,당진교육지원청,공립,면천중학교 운동장 교체,"297,561,000",None,None
191,충청북도,충청북도교육청,공립,충북산업과학고등학교 다목적구장 조성,"769,799,000","5,540",교육청
192,충청북도,청주교육지원청,공립,서경중학교 운동장 보수 및 바닥교체,"602,871,000","3,500",교육청


In [392]:
# 전처리
edu_budget_df['과업명_re'] = edu_budget_df['과업명'].apply(clean_text)
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.replace('초등학교','').str.replace('중학교','').str.replace('고등학교','')
edu_budget_df['과업명_re'] = edu_budget_df['과업명_re'].str.strip()

In [394]:
# 종합쇼핑몰 형식과 통일
edu_budget_df['시군구_re'] = np.where(edu_budget_df['시군구'].str.contains('교육지원청'),
                                   edu_budget_df['시도'] + edu_budget_df['시군구'].str.replace('교육지원청',''), edu_budget_df['시군구']) 

In [396]:
# 교육청, 교육지원청, 학교명 추출
filter_edu1_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu1)],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu1))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu1))]],axis=0)
# edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu1))].shape

filter_edu2_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu2)],
                              edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu2))],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu2))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu2))]],axis=0)

filter_edu3_temp = pd.concat([edu_budget_df[edu_budget_df['시군구_re'].isin(filter_edu3)],
                              edu_budget_df[edu_budget_df['시군구_re'].str.contains(('|').join(filter_edu3))],
                              edu_budget_df[edu_budget_df['과업명'].str.contains(('|').join(filter_edu3))],
                              edu_budget_df[edu_budget_df['과업명_re'].str.contains(('|').join(filter_edu3))]],axis=0)

filtered_edu_budget_df = pd.concat([filter_edu1_temp, filter_edu2_temp, filter_edu3_temp],axis=0).drop_duplicates().reset_index(drop=True)

In [406]:
# 종합쇼핑몰 학교 관련 키워드 빈도수
edu_filter_keywd = ('|').join(edu_mapping_keywd_df[edu_mapping_keywd_df['빈도수'] <= 3]['키워드'].unique())
edu_mapping_keywd_df[edu_mapping_keywd_df['빈도수'] <= 3].sort_values('빈도수',ascending=False) ## 빈도수 평균 2.99

,키워드,빈도수
43,다목적,3
25,운동장환경개선,3
3,교체,3
117,따른,3
47,물품,3
...,...,...
162,화산,1
161,목포부주,1
160,인조잔디구매,1
159,동북,1


In [420]:
# 교육청 제외 데이터
filtered_edu_budget_df_fin = filtered_edu_budget_df[filtered_edu_budget_df['과업명'].str.contains(edu_filter_keywd)].drop_duplicates().reset_index(drop=True)
filtered_edu_budget_df_fin

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re,시군구_re
0,충청남도,보령교육지원청,공립,성주초등학교 운동장 교체,"70,980,000",None,None,성주 운동장 교체,충청남도보령
1,충청남도,보령교육지원청,공립,청파초등학교 운동장 교체,"20,976,000",None,None,청파 운동장 교체,충청남도보령
2,경기도,김포교육지원청,공립,신양중학교 친환경 운동장 조성 사업,None,None,None,신양 친환경 운동장 조성 사업,경기도김포
3,경기도,수원교육지원청,공립,수성중학교 친환경 운동장 조성 사업,None,None,None,수성 친환경 운동장 조성 사업,경기도수원
4,경기도,연천교육지원청,공립,백학중학교 친환경 운동장 조성 사업,None,None,None,백학 친환경 운동장 조성 사업,경기도연천
5,경기도,용인교육지원청,공립,어정초등학교 친환경 운동장 조성 사업,None,None,None,어정 친환경 운동장 조성 사업,경기도용인
6,경기도,용인교육지원청,공립,서원중학교 친환경 운동장 조성 사업,None,None,None,서원 친환경 운동장 조성 사업,경기도용인
7,경기도,파주교육지원청,공립,와석초등학교 친환경 운동장 조성 사업,None,None,None,와석 친환경 운동장 조성 사업,경기도파주
8,경기도,포천교육지원청,공립,영북초등학교 친환경 운동장 조성 사업,None,None,None,영북 친환경 운동장 조성 사업,경기도포천
9,서울특별시,동부교육지원청,공립,상봉중학교(운동장 환경 개선),"680,000,000","4,703",학교,상봉운동장 환경 개선,서울특별시동부


In [418]:
edu_budget_df[~edu_budget_df['과업명'].isin(filtered_edu_budget_df_fin['과업명'])].reset_index(drop=True)

,시도,시군구,구분,과업명,금액,면적,예산집행,과업명_re,시군구_re
0,전북특별자치도,전북특별자치도교육청,None,완주 스포츠클럽 실외야구장,"1,689,820,000",None,None,완주 스포츠클럽 실외야구장,전북특별자치도교육청
1,경기도,경기도교육청,None,친환경 운동장 조성사업비 지원 (30교),"15,000,000,000",None,학교,친환경 운동장 조성사업비 지원 교,경기도교육청
2,전라남도,전라남도교육청,None,운동장 개보수(고2),"1,652,038,000",None,학교,운동장 개보수고,전라남도교육청
3,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 중학교 1교,"14,000,000",None,학교,친환경운동장조성 다목적구장 교,울산광역시교육청
4,울산광역시,울산광역시교육청,None,친환경운동장조성 다목적구장 고등학교 6교,"465,600,000",None,학교,친환경운동장조성 다목적구장 교,울산광역시교육청
...,...,...,...,...,...,...,...,...,...
138,경상남도,의령교육지원청,사립,정곡중학교 체육관 신축(계속비),"38,000,000",None,학교,정곡 체육관 신축계속비,경상남도의령
139,대구광역시,대구광역시교육청,사립,대륜고등학교 축구장 인조잔디 재조성,"578,358,000","7,447",학교,대륜 축구장 인조잔디 재조성,대구광역시교육청
140,울산광역시,울산광역시교육청,사립,친환경운동장조성 다목적구장 중학교 (사립) 1교,"169,141,500",None,학교,친환경운동장조성 다목적구장 사립 교,울산광역시교육청
141,전북특별자치도,전북특별자치도교육청,사립,익산고등학교 인조잔디 조성,"1,050,001,000","7,417",교육청,익산 인조잔디 조성,전북특별자치도교육청
